In [1]:
#Project: 멋진 챗봇 만들기

# NLTK를 활용한 BLEU Score
참고 블루 스코어 설명 https://donghwa-kim.github.io/BLEU.html
BLEU(Bilingual Evaluation Understudy)score란 성과지표로 데이터의 X가 순서정보를 가진 단어들(문장)로 이루어져 있고, y 또한 단어들의 시리즈(문장)로 이루어진 경우에 사용되며, 번역을 하는 모델에 주로 사용된다.

# project 챗봇 만들기

In [2]:
import numpy as np
import tensorflow as tf

import re
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


In [3]:
import os
import pandas as pd
from pandas import DataFrame

data_dir = os.getenv('HOME')+'/aiffel/songys_chatbot/Chatbot_data-master'
data = pd.read_csv("/home/aiffel-dj57/aiffel/songys_chatbot/Chatbot_data-master/ChatbotData .csv")
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [4]:
#데이터 다운로드
#읽어 온 데이터의 질문과 답변을 각각 questions, answers 변수에 나눠서 저장하세요!

df = pd.DataFrame(data) ## 데이터프레임 생성
questions = df['Q']
answers = df['A']

In [5]:
questions

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [6]:
answers

0                      하루가 또 가네요.
1                       위로해 드립니다.
2                     여행은 언제나 좋죠.
3                     여행은 언제나 좋죠.
4                      눈살이 찌푸려지죠.
                   ...           
11818          티가 나니까 눈치가 보이는 거죠!
11819               훔쳐보는 거 티나나봐요.
11820                      설렜겠어요.
11821    잘 헤어질 수 있는 사이 여부인 거 같아요.
11822          도피성 결혼은 하지 않길 바라요.
Name: A, Length: 11823, dtype: object

In [7]:
# 데이터 정제
#영문자의 경우, 모두 소문자로 변환합니다.
#영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.


def preprocess_sentence(sentence, s_token=False, e_token=False):
    sentence = sentence.lower().strip()
    
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^ㄱ-ㅣ가-힣a-zA-Z0-9?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

# 토큰화
토큰화에는 KoNLPy의 mecab 클래스를 사용합니다.

아래 조건을 만족하는 build_corpus() 함수를 구현하세요!

1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.
3. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외합니다.중복되는 문장은 데이터에서 제외합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!
4. 구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.

In [8]:
#q_list = df.Q.str.split('_').str[0].tolist()
#print(q_list[:10])
#a_list = df.A.str.split('_').str[0].tolist() #리스트로 변환해줍니다. 될지는 모르겠어요 
#print(a_list[:10])

#이런 식으로 리스트 형태로 만들어 아래 코드에 하나하나 넣어줬더니 리스트 전체를 도는데 시간이 오래걸려서 에러가 생겼다. 이 방법은 아닌 것 같다

In [9]:
from konlpy.tag import Mecab
tokenizer = Mecab()

In [10]:
def build_corpus(src, tgt, tokenizer, max_len=50):

    pre_src = [preprocess_sentence(x) for x in src]
    pre_tgt = [preprocess_sentence(x) for x in tgt]
    
    #중복제거
    src = []
    tgt = []
    
    for so,tar in zip(pre_src, pre_tgt):
        if so in src :
            continue
        if tar in tgt :
            continue
        src.append(so)
        tgt.append(tar)
    
    que_corpus = []
    ans_corpus = []

    for so, tar in zip(src, tgt):
        src_tokens = tokenizer.morphs(preprocess_sentence(so))
        tgt_tokens = tokenizer.morphs(preprocess_sentence(tar))

        if (len(src_tokens) > 50): continue
        if (len(tgt_tokens) > 50): continue
    
        que_corpus.append(src_tokens)
        ans_corpus.append(tgt_tokens)

    return que_corpus, ans_corpus

In [11]:
que_corpus, ans_corpus = build_corpus(questions, answers, tokenizer)

In [12]:
print('que', que_corpus[:3],'+','ans', ans_corpus[:3])

que [['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다']] + ans [['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.']]


In [13]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import random
#wv = KeyedVectors.load("/home/aiffel-dj57/aiffel/chatbot/ko.bin")
model = Word2Vec.load('/home/aiffel-dj57/aiffel/chatbot/ko.bin')

#gensim을 3.8.1로 다운그레이드 한다.
#pip install gensim==3.8.1

In [16]:
#Lexical Substitution 구현

def lexical_sub_fix(sentence, word2vec):
    import random

    res = ""
    #toks = sentence.split()

    try:
        _from = random.choice(sentence)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in sentence:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res.split()

In [20]:
#Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 
#que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 합니다.
#데이터 30000개!


#노드에 있는 코드를 이해를 못해서 혜미님의 도움을 받았습니다.
# 먼저 question을 변형해보자. 
que_au = []
ans_au = []
for q, a in zip(que_corpus, ans_corpus) :  #qu와 ans corpus 를 받아서 
    q_au = lexical_sub_fix(q, model)       #q_au라는 변수에 lexical_subfix를 수행
    
    # 쌍으로 저장
    que_au.append(q_au)                    #aue_au라는 리스트에 q_au를 넣어준다
    ans_au.append(a)                       #ans_au변수에 ans_corpus의 값들을 담아준다
    #print(que_au)
    a_au = lexical_sub_fix(a,model)        #a_au라는 변수에 lexical_subfix를 수행
    # 쌍으로 저장
    #print(a_au)
    que_au.append(q)                       #que_au에 q를 담아준다
    ans_au.append(a_au)                    #ans_au에 lexical subfix를 수행한 값을 담아준다
new_que_corpus = que_corpus + que_au       #결론적으로 활용할 augumentation이 수행된 corpus : que_corpus + que_au
new_ans_corpus = ans_corpus + ans_au       # ans_corpus + ans_au



['하루', '놀드', '또', '가', '네요', '.']
None
['항해', '은', '언제나', '좋', '죠', '.']
['눈살', '이', '찌푸려', '지', '지요', '.']
['다시', '새로', '사', '는', '게', '괴로움', '편해요', '.']
['많이', '모르', '고', '있', '을', '수', '도', '있', '어요', '.']
['시간', '을', '정해지', '고', '해', '보', '세요', '.']
['자랑', '하', 'ㄴ다는', '자리', '니까요', '.']
['그', '사람', '도', '그럴', '것', '예요', '.']
['혼자', '를', '즐기', '세요', '는데']
['현상금', '은', '다시', '들어올', '거', '예요', '.']
['땀', '을', '식혀', '주', 'ㅂ시오', '.']
['어서', '잊', '고', '감미', '출발', '하', '세요', '.']
['빨리', '집', '에', '돌아가', '서', '끄', '기에', '나오', '세요', '.']
['다음', '달이', '에', '는', '더', '절약', '해', '봐요', '.']
['깨끗', '하', '게', '사세요', '!']
['가장', '확실', '한', '시간', '은', '오늘', '이', '에요', '.', '어제', '와', '내일', '을', '놓', '고', '고민', '하', '열심히', '시간', '을', '낭비', '하', '지', '마세요', '.']
['온', '가족', '이', '모두', '마음', '에', '드', '는', '곳', '으로', '가', '보', 'ㅂ시오', '.']
['저', '를', '만들', '어', '준', '사람', '을', '부모', '님', ',', '저', '블랑', '이야기', '해', '주', '는', '사람', '을', '친구', '로', '생각', '하', '고', '있', '어요']
['더', '가까워질', '일자리', '가', '되',

/home/aiffel-dj57/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


['말', '해', '보', 'ㅂ시오', '.']
['함께', '하', 'ㄴ다면', '서로', '를', '더', '많이', '알', '게', '될', '거', '예요', '.']
['개시', '해', '보', 'ㅂ시오', '.']
['곧', '방학', '그러', '예요', '.']
['방학', '이', '참', '짧', '죠', '는데']
['비롯되', '는', '게', '좋', '겠', '네요', '.']
['세수', '하', '고', '오', '세요', '는데']
['그게', '제일', '중요', '한', '건데요', '는데']
['다음', '부터', '는', '더', '많이', '아세요', '는데']
['공', '의식적', '인', '일', '부터', '하', '세요', '.']
['낮잠', '을', '잠깐', '자', '도', '좋', '아요', '.']
['저', '도', '좋', '아', '해', '주', 'ㅂ시오', '.']
['친구', '들', '이', '보', '고', '싶', '었', '나의', '봐요', '.']
['되', '게끔', '만나', '지', '마세요', '.']
['여러분', '이요', '.']
['당신', '인의', '운', '을', '믿', '어', '보', '세요', '.']
['일', '못', '하', '는', '사람', '이', '없', '으면', '옆', '에', '있', '는', '사람', '이', '더', '힘들', '죠', '.']
['밥', '사', '줄', '친구', '를', '찾', '아', '살펴보', '세요']
['선의', '의', '거짓말', '이', '구불구불', '바라', '요', '.']
['거짓말', '은', '할수록', '늘', '어요', '는데']
['거짓말', '된', '말', '을', '하', '려고', '노력', '해', '보', '세요', '.']
['누구', '나의', '걱정', '은', '있', '어요', '.']
['운동', '을', '해', '보', 'ㅂ시오', '.']
['세

['좋', '겠', '군요', '.']
None
['연예인', '을', '준비', '하', '니', '일반인', '보다', '으며', '예쁘', '겠', '죠', '.']
None
['크', '게', '숨', '한', '번의', '쉬', '어', '보', '세요']
None
['마음', '에', '들', 'ㄴ다면', '줘', '보', '세요', '.']
['잘', '해', '보', 'ㅂ시오', '.']
['그래도', '넘', '을', '가능성', '있', '을', '거', '예요', '.']
['조심', '시키', '세요', '.']
['너무', '낙담', '하', '꼼짝', '마세요', '.']
None
['건강', '을', '위해', '조금', '마다', '드세요', '.']
None
['씹', '는', '식사', '시간', '되', '시', '길', '바랄게요', '.']
['맛있', '지요', '!']
['기다렸', '나', '봐요', '는데']
['조금', '만', '기다리', '면', '다시', '전기', '가', '들어올', '거', '예요', '는데']
['적합', '해요', '.']
['스스로', '단단', '해', '지', '세요', '는데']
['제', '놀드', '드리', '고', '싶', '네요', '.']
['집안', '분위기', '가', '바뀔', '거', '예요', '는데']
['꽃', '대접', '은', '언제나', '좋', '죠', '.']
['솜씨', '가', '좋', '계시', '네요', '.']
['기분', '괜찮', '아', '보이', '세요', '.']
['마음', '의', '안정', '을', '취하', '적기', '좋', '은', '취미', '네요', '.']
None
['거꾸로', '영광송', '드라이플라워', '만들', '어', '보', '세요', '.']
None
['멋진', '대접', '이', '네요', '.']
['센스', '있', '는', '대접', '이', '에요', '.']
['넘겨받', '는', '사람

['고마운', '마음', '을', '전해', '주', 'ㅂ시오', '.']
['적당히', '하', '면', '괜찮', '을', '것', '같', '아요', '.']
['남자', '도', '좋', '은데', '것', '만', '은', '아니', '예요', '.']
['아직', '모르', '겠', '어요', '.', '인공지능', '에', '성별', '을', '만드', '는', '사람', '이', '되', '어', '주', 'ㅂ시오']
['마음', '을', '열', '때', '까지', '설득', '해', '보', '세요', '는데']
['투쟁', '을', '함께', '해', '보', '세요', '.']
['평소', '에', '필요', '한', '것', '생각', '해', '보', '세요', '는데']
['평소', '에', '필요', '했', '던', '게', '좋', '을', '것', '같', '아요', '는데']
['영혼', '에', '나라', '를', '구하', '셨', '나요', '.']
['결단', '은', '빠를수록', '괜찮', '아요', '.']
['거짓말', '적당히', '시키', '세요', '.']
['워낙', '집착', '하', '지', '마세요', '.']
['아무래도', '좀', '깨', '요', '는데']
['바쁠', '때', '힘', '이', '이루어지', '어', '주', '세요', '.']
['그래도', '구박', '시키', '지', '는', '마세요', '.']
['다정', '겠', '네요', '.']
['순간', '실수', '할', '수', '있', '겠', '다', '판단', '이루어지', '면', '용서', '하', '고', '기회', '를', '주', '세요', '.']
['당신', '이', '해', '살펴보', '세요', '.']
['사람', '고쳐', '쓰', '는', '거', '어떠하', '에요', '.']
['더', '잔소리', '해의', '보', '세요', '.']
['다른', '연락', '을', '많이', '하', 

...

['맞춰', '가', '면', '돼요', '는데']
['멋진', '사람', '이', '자', '멋진', '말', '그러', '네요', '.']
['이기', '적', '이', '네요', '는데']
['마린', '라이트', '라고', '생각', '해요', '.']
None
['좋', '아', '하', '면', '신경', '이', '던', '쓰이', '니', '가', '요', '.']
['이제', '상대방', '을', '칭찬', '때때', '차례', '인', '듯', '합니다', '.']
['보', '고', '싶', '으니', '그럴', '만이', '해요', '.']
['안', '햇갈리', '게', '하', '는', '사람', '그러', '좋', '습니다', '.']
['자신', '을', '사랑', '을', '젊은이', '인가', '봅니다', '.']
['코미디', '할', '것', '같', '아요', '.']
['씨익', '웃', '어', '주기도', '세요', '.']
['쪽지', '를', '남겨서', '마음', '을', '전해', '보', '세요', '는데']
['불행', '을', '연습', '하', '도록', '미리', '걱정', '할', '필요', '없', '어요', '.']
['많', '은', '젊은이', '들', '은', '살', '면서', '맞춰', '가', '기', '도', '해요', '.']
['복', '스럽', '게', '먹', '으면', '던', '좋', '죠', '.']
['믿음', '이', '덮이', '기', '전', '에', '는', '불안', '하', '죠', '.']
['깨끗', '한', '마음', '이', '네요', '.']
['지켜보', '는', '게', '좋', '겠', '는데요', '.']
None
['많이', '만나', '보', '는', '게', '좋', '겠', '지만', '연락', '을', '더', '자주', '해', '보', 'ㅂ시오', '.']
['물리', '적', '인', '거리', '가', '문제', '놀드', '되

None
['의미', '주어지', '가', '특히', '사람', '을', '기대', '하', '게', '만들', '기', '때문', '이', '에요', '.']
['사랑', '의', '콩깍지', '가', '적히', '었', '네요', '.']
None
None
['그래서', '보통', '확실', '할', '때', '고백', '을', '시키', '죠', '.']
['충분히', '그렇게', '판단', '할', '수', '있', '어요', '.']
['있', '습니다', '는데']
['충분', '지칠', '만', '해요', '.']
['정말', '귀찮', '다면', '답장', '시키', '지', '않', '을', '거', '예요', '.']
['그렇게', '교제', '를', '이', '어', '나가', '는', '것', '도', '방법', '이', '에요', '.']
None
['말', '해의', '보', '는', '게', '우선', '이', '에요', '.']
['음료수', '라도', '주', '면서', '자연', '스럽', '게', '근황', '을', '물', '어서', '봐요', '.']
['맨날', '하', '지', '말', '고', '조금', '은', '여지', '를', '줘', '살펴보', '세요', '.']
['뭐', '하', '는지', '바닷물', '어', '보', '세요', '.']
['먼저', '다가가', 'ㄴ다는', '태도', '좋', '아요', '.']
['적극', '적', '이', '면', '도리어', '더', '좋', '을', '것', '같', '아요', '.']
['각자', '의', '방식', '그러', '있', '으니', '존중', '해야', '해요', '.']
['먼저', '연락', '해도', '힘들', '어', '말', '은데요', '.']
['빨리', '연락', '해서', '대화', '를', '이', '어', '나가', '는', '게', '더', '좋', '습니다', '.']
None
['궁금', '할', '만', '해요', '는데

['짝사랑', '을', '하', '다', '지치', '면', '그럴', '수', '있', '어요', '는데']
None
None
None
['그', '가', '무엇', '을', '좋아하', '다면', '주변', '에', '물', '어', '봐요', '.']
['신경', '쓰일', '거', '라', '판단', '해요', '.']
['살짝', '티', '내고', '려는', '거', '라면', '찬성', '합니다', '.']
['공통', '관심사', '을', '찾아보', '세요', '.']
['필요', '한', '도록', '없', '는지', '물', '어', '보', '면서', '대화', '를', '이', '어', '나가', '세요', '.']
['더', '나', '은', '젊은이', '많', '아요', '.']
['시간', '있', '냐고', '물어봐', '보', 'ㅂ시오', '.']
['밤', '데이트', '신청', '즐겁', '네요', '.']
['짐작', '시키', '는', '건', '당신', '을', '더', '슬프', '게', '만들', '어요', '.']
['나', '의', '존재', '를', '빨리', '인지', '시켜', '보', '세요', '.']
['사랑', '으로', '이어질', '수', '있', '는데요', '.']
['경쟁자', '를', '없애', '기', '위해', '그게', '나을', '수', '도', '있', '는데요', '.']
['질투', '난다고', '살짝', '말', '해의', '봐요', '.']
['어떤', '계기', '가', '있', '었', '를', '거', '예요', '.']
None
['방해', '가', '될', '수', '도', '있', '지만', '옆', '에서', '응원', '해', '주', '면', '든든', '할', '방법', '같', '아요', '.']
['입장', '을', '확실히', '하', 'ㄴ다는', '편', '이', '좋', '겠', '어요', '.']
['상황', '이', '복잡', '하', '군요

['짝사랑', '이', '엄청', '만드', '네요', '.']
['짝사랑', '은', '자신', '을', '외롭', '게', '만들', '어요', '는데']
['그리워하', '상대', '생각', '에', '설레', '는', '것', '.']
['사랑', '이', '었', '으니까요', '는데']
['짝사랑', '에', '기분', '이', '오락가락', '하', '지요', '.']
['많이', '만날', '수', '있', '는', '계기', '를', '만들', '어', '살펴보', '세요', '.']
['취체역', '는', '가', '더라도', '연락', '은', '지속', '해', '보', '세요', '.']
None
['정말', '맘고생', '많', '았', '는데요', '.']
['다가가', '는', '만큼', '확률', '이', '높', '아서', '질', '거', '예요', '.']
None
['사랑', '앞', '이에', '작아지', '곤', '해요', '.']
['꼭', '잘', '될', '것', '예요', '.']
['그것', '정도', '마음', '아픈', '일', '도', '없', '겠', '네요', '.']
None
['직접', '물', '어', '보', '는', '것', '도', '좋', '은', '방법', '이', '에요', '는데']
['용기', '를', '내', '서', '고백', '해', '살펴보', '는', '건', '어떨까', '요', '.']
['사랑', '그러', '내', '맘', '같', '지', '않', '아서', '그렇게', '되', '나', '봐요', '.']
['판단', '처럼', '쉽', '지', '않', '지요', '.']
['끝', '을', '알리', '는', '것', '도', '필요', '합니다', '.']
['먼저', '연락', '해', '보', 'ㄴ다는', '것', '도', '좋', '을', '것', '같', '습니다', '.']
['좋', '아', '하', '는', '건', '본인', '의', '평화',

['결심', '은', '빠르', '면', '빠를수록', '괜찮', '아요', '.']
['법의', '적', '부부', '가', '된', '걸', '축하', '해요', '!']
['힘들', '면', '잠시', '놓', '는', '것', '도', '방법', '이', 'ㄴ가요', '.']
['외로운', '상태', '인', '거', '똑같', '아요', '.']
['적극', '적', '으로', '꼬셔', '보', 'ㅂ시오', '.']
['손수건', '추진', '할게요', '.']
None
['사랑', '은', '더', '잘', '하', '게', '되', '지', '않', '고', '다시', '영', '에서', '시작', '하', '니', '모두', '본격적', '이', '겠', '죠', '.']
['거부', '의', '뜻', '은', '아니', '었', '나요', '.']
['혼자', '가', '편하', '다는', '것', '이', '거절', '의', '뜻', '은', '아니', '었', '을까요', '는데']
['화장실', '불빛', '으로', '좀', '멀리', '가리', '고', '해의', '보', '세요', '.']
['자주', '있', '어요', '.']
None
['설렜', '를', '텐데', '아쉽', '겠', '어요', '.']
None
None
['정신', '없이', '바쁠', '지도', '몰라요', '.', '조금', '만이', '더', '기다려', '보', '고', '물', '어', '보', '는', '게', '좋', '을', '것', '같', '아요', '.']
['진심', '으로', '들키', '보', '세요', '.']
['티', '가', '나', '니까', '만지', '가', '보이', '는', '거', '죠', '!']
['설렜', '겠', '는데요', '.']
['잘', '헤어질', '수', '있', '는', '사이', '여부', '인', '거', '같', '아요', '는데']


In [28]:
que_au[:10] #아래 코드에서 중간중간 있는 none값을 제거해준다

[['12', '시', '땡', '캐치'],
 ['12', '시', '땡', '!'],
 None,
 ['1', '지망', '학교', '떨어졌', '어'],
 None,
 ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
 ['ppl', '심하', '카나'],
 ['ppl', '심하', '네'],
 None,
 ['sd', '카드', '망가졌', '어']]

In [29]:
final_que_corpus = []
final_ans_corpus = []

for q, a in zip(new_que_corpus, new_ans_corpus) :    #new que와 new_ans안의 값들 중
    if q is None :                                   #q가 none값이면 제거 : contunue
        continue
    if a is None :                                   #a가 none값이면 제거 : continue
        continue
    final_que_corpus.append(q)                       #그 외의 나머지 값들을 각각의 final corpus에 담아준다
    final_ans_corpus.append(a)

In [31]:
print(len(final_que_corpus))
print(len(final_ans_corpus))

21239
21239


In [32]:
#augumentation 후에 전처리와 벡터화를 거쳐준다
#spm 모델 활용


def generate_tokenizer(corpus,
                       vocab_size,
                       lang="ko",
                       pad_id=0,
                       bos_id=1,
                       eos_id=2,
                       unk_id=3):
    file = "/home/aiffel-dj15/aiffel/transformer_chatbot/%s_corpus.txt" % lang
    model = "/home/aiffel-dj15/aiffel/transformer_chatbot/%s_spm" % lang
    
    with open(file, 'w') as f:
        for row in corpus: f.write(str(row) + '\n')

    import sentencepiece as spm
    spm.SentencePieceTrainer.Train(
        '--input=%s --model_prefix=%s --vocab_size=%d'\
        % (file, model, vocab_size) + \
        '--pad_id==%d --bos_id=%d --eos_id=%d --unk_id=%d'\
        % (pad_id, bos_id, eos_id, unk_id)
    )
    #print(model)
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)

    return tokenizer